In [ ]:
# export
from nbdev.imports import *
from nbdev.sync import *
from nbdev.export import *
from nbdev.showdoc import *

from nbconvert.preprocessors import ExecutePreprocessor, Preprocessor
from nbconvert import HTMLExporter,MarkdownExporter
import traitlets

In [ ]:
# default_exp export2html
# default_cls_lvl 3

# Converting notebooks to html

> The functions that transform the dev notebooks in the documentation of the library

## Preprocessing notebook

### Cell processors

In [ ]:
#export
def remove_widget_state(cell):
    "Remove widgets in the output of `cells`"
    if cell['cell_type'] == 'code' and 'outputs' in cell:
        cell['outputs'] = [l for l in cell['outputs']
                           if not ('data' in l and 'application/vnd.jupyter.widget-view+json' in l.data)]
    return cell

In [ ]:
#export
# Matches any cell that has a `show_doc` or an `#export` in it
_re_cell_to_hide = r's*show_doc\(|^\s*#\s*export\s+'

In [ ]:
#export
def hide_cells(cell):
    "Hide `cell` that need to be hidden"
    if check_re(cell, _re_cell_to_hide):  cell['metadata'] = {'hide_input': True}
    return cell

In [ ]:
for source in ['show_doc(read_nb)', '# export\nfrom local.core import *']:
    cell = {'cell_type': 'code', 'source': 'show_doc(read_nb)'}
    cell1 = hide_cells(cell.copy())
    assert 'metadata' in cell1
    assert 'hide_input' in cell1['metadata']
    assert cell1['metadata']['hide_input']

cell = {'cell_type': 'code', 'source': '# exports\nfrom local.core import *'}
test_eq(hide_cells(cell.copy()), cell)

In [ ]:
#export
# Matches any line containing an #exports
_re_exports = re.compile(r'^#\s*exports[^\n]*\n')

In [ ]:
#export
def clean_exports(cell):
    "Remove exports flag from `cell`"
    cell['source'] = _re_exports.sub('', cell['source'])
    return cell

In [ ]:
cell = {'cell_type': 'code', 'source': '# exports\nfrom local.core import *'}
test_eq(clean_exports(cell.copy()), {'cell_type': 'code', 'source': 'from local.core import *'})
cell = {'cell_type': 'code', 'source': '# exports core\nfrom local.core import *'}
test_eq(clean_exports(cell.copy()), {'cell_type': 'code', 'source': 'from local.core import *'})

In [ ]:
#export
def treat_backticks(cell):
    "Add links to backticks words in `cell`"
    if cell['cell_type'] == 'markdown': cell['source'] = add_doc_links(cell['source'])
    return cell

In [ ]:
cell = {'cell_type': 'markdown', 'source': 'This is a `DocsTestClass`'}
test_eq(treat_backticks(cell), {'cell_type': 'markdown',
    'source': 'This is a [`DocsTestClass`](/export.html#DocsTestClass)'})

In [ ]:
#export
_re_nb_link = re.compile(r"""
# Catches any link to a local notebook and keeps the title in group 1, the link without .ipynb in group 2
\[          # Opening [
([^\]]*)    # Catching group for any character except ]
\]\(        # Closing ], opening (
([^http]    # Catching group that must not begin by html (local notebook)
[^\)]*)     # and containing anything but )
.ipynb\)    # .ipynb and closing )
""", re.VERBOSE)

In [ ]:
# export
def convert_links(cell):
    "Convert the .ipynb links to .html"
    if cell['cell_type'] == 'markdown':
        cell['source'] = _re_nb_link.sub(r'[\1](\2.html)', cell['source'])
    return cell

In [ ]:
cell = {'cell_type': 'markdown', 'source': "This is a link to a [notebook](01_core.ipynb)."}
test_eq(convert_links(cell), {'cell_type': 'markdown', 
                               'source': "This is a link to a [notebook](01_core.html)."})
cell = {'cell_type': 'markdown', 'source': "This is a link to a [page](01_core.html)."}
test_eq(convert_links(cell.copy()), cell)
cell = {'cell_type': 'markdown', 'source': "This is a link to an [external nb](http://01_core.ipynb)."}
test_eq(convert_links(cell.copy()), cell)

In [ ]:
#export
_re_block_notes = re.compile(r"""
# Catches any pattern > Title: content with title in group 1 and content in group 2
>\s*      # > followed by any number of whitespace
([^:]*)   # Catching group for any character but :
:\s*      # : then any number of whitespace
([^\n]*)  # Catching group for anything but a new line character
(?:\n|$)  # Non-catching group for either a new line or the end of the text
""", re.VERBOSE)

In [ ]:
#export
def add_jekyll_notes(cell):
    "Convert block quotes to jekyll notes in `cell`"
    t2style = {'Note': 'info', 'Warning': 'danger', 'Important': 'warning'}
    def _inner(m):
        title,text = m.groups()
        style = t2style.get(title, None)
        if style is None: return f"> {m.groups()[0]}: {m.groups()[1]}"
        res = f'<div markdown="span" class="alert alert-{style}" role="alert">'
        return res + f'<i class="fa fa-{style}-circle"></i> <b>{title}: </b>{text}</div>'
    if cell['cell_type'] == 'markdown':
        cell['source'] = _re_block_notes.sub(_inner, cell['source'])
    return cell

Supported styles are `Warning`, `Note` and `Important`:

> Warning: There will be no second warning!

> Important: Pay attention! This is important.

> Note: Take note of this.

In [ ]:
for w,s in zip(['Warning', 'Note', 'Important', 'Bla'], ['danger', 'info', 'warning', 'info']):
    cell = {'cell_type': 'markdown', 'source': f"> {w}: This is my final {w.lower()}!"}
    res = f'<div markdown="span" class="alert alert-{s}" role="alert">'
    res += f'<i class="fa fa-{s}-circle"></i> <b>{w}: </b>This is my final {w.lower()}!</div>'
    if w != 'Bla': test_eq(add_jekyll_notes(cell), {'cell_type': 'markdown', 'source': res})
    else: test_eq(add_jekyll_notes(cell), cell)

In [ ]:
#export
_re_image = re.compile(r"""
# Catches any image file used, either with `![alt](image_file)` or `<img src="image_file">`
^(!\[         #   Beginning of line (since re.MULTILINE is passed) followed by ![ in a catching group
[^\]]*        #   Anything but ]
\]\()         #   Closing ] and opening (, end of the first catching group
([^\)]*)      #   Catching block with any character but )
(\))          #   Catching group with closing )
|             # OR
(<img\ src=") #   Catching group with <img src="
([^"]*)       #   Catching block with any character except "
(")           #   Catching group with closing "
""", re.MULTILINE | re.VERBOSE)

In [ ]:
#export
def copy_images(cell, fname, dest):
    "Copy images referenced in `cell` from `fname` parent folder to `dest` folder"
    if cell['cell_type'] == 'markdown' and _re_image.search(cell['source']):
        grps = _re_image.search(cell['source']).groups()
        src = grps[1] or grps[4]
        os.makedirs((Path(dest)/src).parent, exist_ok=True)
        shutil.copy(Path(fname).parent/src, Path(dest)/src)
    return cell

In [ ]:
dest_img = Config().doc_path/'images'/'logo.png' 
dest_bak = Config().doc_path/'images'/'logo.bak'
if dest_img.exists(): shutil.move(dest_img, dest_bak)
for text in ['Text\n![Alt](images/logo.png)', 
             'Text\n<img src="images/logo.png" alt="Logo" style="width: 100%; height: auto;"/>']:
    cell = {'cell_type': 'markdown', 'source': text}
    cell1 = copy_images(cell, Path('01_export.ipynb'), Config().doc_path)
    #Function doesn't touch cell
    test_eq(cell, cell1)
    #Image has been copied
    assert dest_img.exists()
    os.remove(dest_img)
if dest_bak.exists(): shutil.move(dest_bak, dest_img)

In [ ]:
path = Path('images').absolute()

In [ ]:
path.parts

('/', 'home', 'sgugger', 'git', 'nbdev', 'nbs', 'images')

In [ ]:
#export
def _relative_to(path1, path2):
    p1,p2 = Path(path1).absolute().parts,Path(path2).absolute().parts
    i=0
    while i <len(p1) and i<len(p2) and p1[i] == p2[i]: i+=1
    p1,p2 = p1[i:],p2[i:]
    return os.path.sep.join(['..' for _ in p2] + list(p1))

In [ ]:
#hide
test_eq(_relative_to(Path('images/logo.png'), Config().doc_path), '../nbs/images/logo.png')
test_eq(_relative_to(Path('images/logo.png'), Config().doc_path.parent), 'nbs/images/logo.png')

In [ ]:
#export
def adapt_img_path(cell, fname, dest):
    "Copy images referenced in `cell` from `fname` parent folder to `dest` folder"
    def _rep(m):
        gps = m.groups()
        start,img,end = gps[:3] if gps[0] is not None else gps[3:]
        new_img = _relative_to(fname.parent/img, dest)
        return f'{start}{new_img}{end}'
    if cell['cell_type'] == 'markdown': cell['source'] = _re_image.sub(_rep, cell['source'])
    return cell

In [ ]:
cell = {'cell_type': 'markdown', 'source': 'Text\n![Alt](images/logo.png)'}
cell1 = adapt_img_path(cell, Path('01_export.ipynb'), Path('.').absolute().parent)
test_eq(cell1['source'], 'Text\n![Alt](nbs/images/logo.png)')

cell = {'cell_type': 'markdown', 
        'source': 'Text\n<img src="images/logo.png" alt="Logo" style="width: 100%; height: auto;"/>'}
cell1 = adapt_img_path(cell, Path('01_export.ipynb'), Path('.').absolute().parent)
test_eq(cell1['source'], 'Text\n<img src="nbs/images/logo.png" alt="Logo" style="width: 100%; height: auto;"/>')

In [ ]:
#export
#Matches any cell with #hide or #default_exp or #default_cls_lvl
_re_cell_to_remove = re.compile(r'^\s*#\s*(hide|default_exp|default_cls_lvl)\s+')

In [ ]:
#export
def remove_hidden(cells):
    "Remove in `cells` the ones with a flag `#hide` or `#default_exp`"
    return [c for c in cells if _re_cell_to_remove.search(c['source']) is None]

In [ ]:
cells = [{'cell_type': 'code', 'source': source} for source in [
    '# export\nfrom local.core import *', 
    '# hide\nfrom local.core import *',
    '#exports\nsuper code',
    '#default_exp notebook.export',
    'show_doc(read_nb)',
    '#default_cls_lvl 3']] + [{'cell_type': 'markdown', 'source': source} for source in [
    'nice', '#hide\n\nto hide']]
         
cells1 = remove_hidden(cells)
test_eq(len(cells1), 4)
test_eq(cells1[0], cells[0])
test_eq(cells1[1], cells[2])
test_eq(cells1[2], cells[4])
test_eq(cells1[3], cells[6])

In [ ]:
#export
_re_default_cls_lvl = re.compile(r"""
^               # Beginning of line (since re.MULTILINE is passed)
\s*\#\s*        # Any number of whitespace, #, any number of whitespace
default_cls_lvl # default_cls_lvl
\s*             # Any number of whitespace
(\d*)           # Catching group for any number of digits
\s*$            # Any number of whitespace and end of line (since re.MULTILINE is passed)
""", re.IGNORECASE | re.MULTILINE | re.VERBOSE)

In [ ]:
# export
def find_default_level(cells):
    "Find in `cells` the default export module."
    for cell in cells:
        tst = check_re(cell, _re_default_cls_lvl)
        if tst: return int(tst.groups()[0])
    return 2

In [ ]:
tst_nb = read_nb('00_export.ipynb')
test_eq(find_default_level(tst_nb['cells']), 3)

In [ ]:
#export
#Find a cell with #export(s)
_re_export = re.compile(r'^\s*#\s*exports?\s*', re.IGNORECASE | re.MULTILINE)
_re_show_doc = re.compile(r"""
# First one catches any cell with a #export or #exports, second one catches any show_doc and get the first argument in group 1
show_doc     # show_doc
\s*\(\s*     # Any number of whitespace, opening (, any number of whitespace
([^,\)\s]*)  # Catching group for any character but a comma, a closing ) or a whitespace
[,\)\s]      # A comma, a closing ) or a whitespace
""", re.MULTILINE | re.VERBOSE)

In [ ]:
#export
def _show_doc_cell(name, cls_lvl=None):
    return {'cell_type': 'code',
            'execution_count': None,
            'metadata': {},
            'outputs': [],
            'source': f"show_doc({name}{'' if cls_lvl is None else f', default_cls_level={cls_lvl}'})"}

def add_show_docs(cells, cls_lvl=None):
    "Add `show_doc` for each exported function or class"
    documented = [_re_show_doc.search(cell['source']).groups()[0] for cell in cells
                  if cell['cell_type']=='code' and _re_show_doc.search(cell['source']) is not None]
    res = []
    for cell in cells:
        res.append(cell)
        if check_re(cell, _re_export):
            names = export_names(cell['source'], func_only=True)
            for n in names:
                if n not in documented: res.append(_show_doc_cell(n, cls_lvl=cls_lvl))
    return res

In [ ]:
for i,cell in enumerate(tst_nb['cells']):
    if cell['source'].startswith('#export\ndef read_nb'): break
tst_cells = [c.copy() for c in tst_nb['cells'][i-1:i+1]]
added_cells = add_show_docs(tst_cells, cls_lvl=3)
test_eq(len(added_cells), 3)
test_eq(added_cells[0], tst_nb['cells'][i-1])
test_eq(added_cells[1], tst_nb['cells'][i])
test_eq(added_cells[2], _show_doc_cell('read_nb', cls_lvl=3))
test_eq(added_cells[2]['source'], 'show_doc(read_nb, default_cls_level=3)')

#Check show_doc isn't added if it was already there.
tst_cells1 = [{'cell_type':'code', 'source': '#export\ndef my_func(x):\n    return x'},
              {'cell_type':'code', 'source': 'show_doc(my_func)'}]
test_eq(add_show_docs(tst_cells1), tst_cells1)
tst_cells1 = [{'cell_type':'code', 'source': '#export\ndef my_func(x):\n    return x'},
              {'cell_type':'markdown', 'source': 'Some text'},
              {'cell_type':'code', 'source': 'show_doc(my_func, title_level=3)'}]
test_eq(add_show_docs(tst_cells1), tst_cells1)

In [ ]:
#export 
_re_fake_header = re.compile(r"""
# Matches any fake header (one that ends with -)
\#+    # One or more #
\s+    # One or more of whitespace
.*     # Any char
-\s*   # A dash followed by any number of white space
$      # End of text
""", re.VERBOSE)

In [ ]:
# export
def remove_fake_headers(cells):
    "Remove in `cells` the fake header"
    return [c for c in cells if c['cell_type']=='code' or _re_fake_header.search(c['source']) is None]

In [ ]:
cells = [{'cell_type': 'markdown',
          'metadata': {},
          'source': '### Fake-'}] + tst_nb['cells'][:10]
cells1 = remove_fake_headers(cells)
test_eq(len(cells1), len(cells)-1)
test_eq(cells1[0], cells[1])

In [ ]:
# export
def remove_empty(cells):
    "Remove in `cells` the empty cells"
    return [c for c in cells if len(c['source']) >0]

### Grabbing metada

In [ ]:
#export 
_re_title_summary = re.compile(r"""
# Catches the title and summary of the notebook, presented as # Title > summary, with title in group 1 and summary in group 2
^\s*       # Beginning of text followe by any number of whitespace
\#\s+      # # followed by one or more of whitespace
([^\n]*)   # Catching group for any character except a new line
\n+        # One or more new lines
>\s*       # > followed by any number of whitespace
([^\n]*)   # Catching group for any character except a new line
""", re.VERBOSE)

_re_properties = re.compile(r"""
^-\s+      # Beginnig of a line followed by - and at least one space
(.*?)      # Any pattern (shortest possible)
\s*:\s*    # Any number of whitespace, :, any number of whitespace
(.*?)$     # Any pattern (shortest possible) then end of line
""", re.MULTILINE | re.VERBOSE)

In [ ]:
# export
def get_metadata(cells):
    "Find the cell with title and summary in `cells`."
    for i,cell in enumerate(cells):
        if cell['cell_type'] == 'markdown':
            match = _re_title_summary.match(cell['source'])
            if match:
                cells.pop(i)
                attrs = {k:v for k,v in _re_properties.findall(cell['source'])}
                return {'keywords': 'fastai',
                        'summary' : match.groups()[1],
                        'title'   : match.groups()[0],
                        **attrs}
                
    return {'keywords': 'fastai',
            'summary' : 'summary',
            'title'   : 'Title'}

In [ ]:
tst_nb = read_nb('00_export.ipynb')
test_eq(get_metadata(tst_nb['cells']), {
    'keywords': 'fastai',
    'summary': 'The functions that transform notebooks in a library',
    'title': 'Converting notebooks to modules'})
#The cell with the metada is poped out, so if we do it a second time we get the default.
test_eq(get_metadata(tst_nb['cells']), {'keywords': 'fastai',
            'summary' : 'summary',
            'title'   : 'Title'})

## Executing show_doc cells

In [ ]:
#export
#Catches any cell with a show_doc or an import from local
_re_cell_to_execute = re.compile(r"^\s*show_doc\(([^\)]*)\)|^from " + Config().lib_name + "\.", re.MULTILINE)

In [ ]:
# export
class ExecuteShowDocPreprocessor(ExecutePreprocessor):
    "An `ExecutePreprocessor` that only executes `show_doc` and `import` cells"
    def preprocess_cell(self, cell, resources, index):
        if 'source' in cell and cell['cell_type'] == "code":
            if _re_cell_to_execute.search(cell['source']):
                return super().preprocess_cell(cell, resources, index)
        return cell, resources

In [ ]:
# export
def _import_show_doc_cell(mod=None):
    "Add an import show_doc cell."
    source = f"#export\nfrom nbdev.showdoc import show_doc"
    if mod:  source += f"\nfrom {Config().lib_name}.{mod} import *"
    return {'cell_type': 'code',
            'execution_count': None,
            'metadata': {'hide_input': True},
            'outputs': [],
            'source': source}

def execute_nb(nb, mod=None, metadata=None, show_doc_only=True):
    "Execute `nb` (or only the `show_doc` cells) with `metadata`"
    nb['cells'].insert(0, _import_show_doc_cell(mod))
    ep_cls = ExecuteShowDocPreprocessor if show_doc_only else ExecutePreprocessor
    ep = ep_cls(timeout=600, kernel_name='python3')
    metadata = metadata or {}
    pnb = nbformat.from_dict(nb)
    ep.preprocess(pnb, metadata)
    return pnb

In [ ]:
fake_nb = {k:v for k,v in tst_nb.items() if k != 'cells'}
fake_nb['cells'] = [tst_nb['cells'][0].copy()] + added_cells
fake_nb = execute_nb(fake_nb, mod='export')
assert len(fake_nb['cells'][-1]['outputs']) > 0

## Conversion

In [ ]:
__file__ = Config().lib_path/'export2html.py'

In [ ]:
# export
def _exporter(markdown=False):
    cfg = traitlets.config.Config()
    exporter = (HTMLExporter,MarkdownExporter)[markdown](cfg)
    exporter.exclude_input_prompt=True
    exporter.exclude_output_prompt=True
    exporter.template_file = ('jekyll.tpl','jekyll-md.tpl')[markdown]
    exporter.template_path.append(str(Path(__file__).parent/'templates'))
    return exporter

In [ ]:
# export
process_cells = [remove_fake_headers, remove_hidden, remove_empty]
process_cell  = [hide_cells, remove_widget_state, add_jekyll_notes, convert_links]

In [ ]:
# export
def convert_nb(fname):
    "Convert a notebook `fname` to html file in `dest_path`."
    fname = Path(fname).absolute()
    nb = read_nb(fname)
    cls_lvl = find_default_level(nb['cells'])
    mod = find_default_export(nb['cells'])
    nb['cells'] = compose(*process_cells,partial(add_show_docs, cls_lvl=cls_lvl))(nb['cells'])
    nb['cells'] = [compose(partial(copy_images, fname=fname, dest=Config().doc_path), *process_cell, treat_backticks)(c)
                    for c in nb['cells']]
    dest_name = '.'.join(fname.with_suffix('.html').name.split('_')[1:])
    meta_jekyll = get_metadata(nb['cells'])
    meta_jekyll['nb_path'] = str(fname.relative_to(Config().lib_path.parent))
    nb = execute_nb(nb, mod=mod)
    nb['cells'] = [clean_exports(c) for c in nb['cells']]
    with open(Config().doc_path/dest_name,'w') as f:
        f.write(_exporter().from_notebook_node(nb, resources=meta_jekyll)[0])

In [ ]:
convert_nb('00_export.ipynb')

In [ ]:
# export
def convert_all(fname=None, force_all=False):
    "Convert all notebooks in `path` to html files in `dest_path`."
    changed_cnt = 0
    if fname is None:
        files = [f for f in Config().nbs_path.glob('*.ipynb') if not f.name.startswith('_')]
    else: files = glob.glob(fname)
    for fname in files:
        # only rebuild modified files
        fname_out = Config().doc_path/'.'.join(fname.with_suffix('.html').name.split('_')[1:])
        if not force_all and fname_out.exists() and os.path.getmtime(fname) < os.path.getmtime(fname_out):
            continue
        print(f"converting: {fname.name} => {fname_out.name}")
        changed_cnt += 1
        try: convert_nb(fname)
        except Exception as e: print(e)
    if changed_cnt==0: print("No notebooks were modified")

In [ ]:
#hide
#convert_all(force_all=True)

In [ ]:
# export
def convert_md(fname, dest_path):
    "Convert a notebook `fname` to a markdown file in `dest_path`."
    fname = Path(fname).absolute()
    nb = read_nb(fname)
    meta_jekyll = get_metadata(nb['cells'])
    nb['cells'] = compose(*process_cells)(nb['cells'])
    nb['cells'] = [compose(partial(copy_images, fname=fname, dest=dest_path), *process_cell)(c) for c in nb['cells']]
    fname = Path(fname).absolute()
    dest_name = fname.with_suffix('.md').name
    exp = _exporter(markdown=True)
    with (Path(dest_path)/dest_name).open('w') as f:
        f.write(exp.from_notebook_node(nb, resources=meta_jekyll)[0])

In [ ]:
convert_md('99_index.ipynb', Path('.').absolute().parent)

## Export-

In [ ]:
#hide
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 09_index.ipynb.
